Make directories according to the barcode file, one for each sample

In [ ]:
mkdir $(awk -F"," '{print $1}' /local/home/sarahgw/WGS/Sequences/Pool3/Pool3_Barcodes.csv)

In [5]:
!ls /local/home/sarahgw/WGS/Sequences/Pool3/

NJ_E_01  NY_E_28  NY_U_10  NY_U_15	       VT_E_27	VT_U_01  VT_U_29
NJ_E_02  NY_E_29  NY_U_11  NY_U_16	       VT_E_28	VT_U_02
NJ_E_04  NY_E_30  NY_U_12  Pool3_Barcodes.csv  VT_E_29	VT_U_03
NY_E_27  NY_U_09  NY_U_14  VT_E_26	       VT_E_30	VT_U_27


In [6]:
!rm /local/home/sarahgw/WGS/Sequences/Pool3/Pool3_Barcodes.csv

In [7]:
!ls /local/home/sarahgw/WGS/Sequences/Pool3/

NJ_E_01  NY_E_27  NY_E_30  NY_U_11  NY_U_15  VT_E_27  VT_E_30  VT_U_03
NJ_E_02  NY_E_28  NY_U_09  NY_U_12  NY_U_16  VT_E_28  VT_U_01  VT_U_27
NJ_E_04  NY_E_29  NY_U_10  NY_U_14  VT_E_26  VT_E_29  VT_U_02  VT_U_29


remove samples that didnt have high enough concentrations

In [9]:
!rmdir /local/home/sarahgw/WGS/Sequences/Pool3/NY_U_09/

In [10]:
!rmdir /local/home/sarahgw/WGS/Sequences/Pool3/VT_E_27/

In [11]:
!ls /local/home/sarahgw/WGS/Sequences/Pool3/

NJ_E_01  NY_E_27  NY_E_30  NY_U_12  NY_U_16  VT_E_29  VT_U_02  VT_U_29
NJ_E_02  NY_E_28  NY_U_10  NY_U_14  VT_E_26  VT_E_30  VT_U_03
NJ_E_04  NY_E_29  NY_U_11  NY_U_15  VT_E_28  VT_U_01  VT_U_27


Copy files into matching directories

In [ ]:
for dir in $(ls); do cd $dir; cp /local/shared/pinsky_lab/sequencing/hiseq_2017_04_20_SEQ19/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-3-from-flowcell-HJFCHBCXY-on-2017-04-27_Read_1_passed_filter.fastq.gz /local/shared/pinsky_lab/sequencing/hiseq_2017_04_20_SEQ19/"$dir"_Demultiplexed-fastq-fileset-sample-Pool-3-from-flowcell-HJFCHBCXY-on-2017-04-27_Read_4_passed_filter.fastq.gz . ; cd ..; done

In [21]:
!ls /local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_02

NJ_E_02_Demultiplexed-fastq-fileset-sample-Pool-3-from-flowcell-HJFCHBCXY-on-2017-04-27_Read_1_passed_filter.fastq
NJ_E_02_Demultiplexed-fastq-fileset-sample-Pool-3-from-flowcell-HJFCHBCXY-on-2017-04-27_Read_4_passed_filter.fastq


In [ ]:
gunzip */*.fastq.gz

In [ ]:
!ls /local/home/sarahgw/WGS/Sequences/Pool3/VT_U_29

Lowest number of sequences

In [33]:
!grep -c '@' /local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_01/NJ_E_01_Demultiplexed-fastq-fileset-sample-Pool-3-from-flowcell-HJFCHBCXY-on-2017-04-27_Read_4_passed_filter.fastq

7769124


Run fastuniq on unzipped files, making the forfastuniq file each time.

In [ ]:
for dir in $(ls); do cd $dir; ls > /tmp/forFastUniq.txt; /local/home/sarahgw/FastUniq/fastuniq -i /tmp/forFastUniq.txt -t q -o $(pwd)/"$dir"_FU1.fastq -p $(pwd)/"$dir"_FU2.fastq; cd ..; done

Trim

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/Trimmomatic-0.36/trimmomatic-0.36.jar PE -phred33 $(pwd)/"$dir"_FU1.fastq $(pwd)/"$dir"_FU2.fastq $(pwd)/"$dir"_FU1_trim_p.fastq $(pwd)/"$dir"_FU1_trim_up.fastq $(pwd)/"$dir"_FU2_trim_p.fastq $(pwd)/"$dir"_FU2_trim_up.fastq ILLUMINACLIP:/local/home/sarahgw/Trimmomatic-0.36/adapters/NexteraPE-PE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:30; cd ..; done

In [1]:
!grep -c '@' /local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_01/NJ_E_01_FU1_trim_p.fastq

6962546


In [4]:
!grep -c '@' /local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_01/NJ_E_01_FU2_trim_p.fastq

6962546


Run Fastqscreen on paired and unpaired samples, first just tag them, then rerun up with filter 0000, then run pfilter on paired tagged reads. Move tagged reads to /Scratch

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/fastq_screen_v0.9.5/fastq_screen --subset 0 --tag --aligner  Bowtie2 $(pwd)/"$dir"_FU1_trim_p.fastq $(pwd)/"$dir"_FU2_trim_p.fastq; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/fastq_screen_v0.9.5/fastq_screen --subset 0 --tag --aligner  Bowtie2 $(pwd)/"$dir"_FU1_trim_up.fastq $(pwd)/"$dir"_FU2_trim_up.fastq; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/fastq_screen_v0.9.5/fastq_screen --subset 0 --tag --filter 00000 --aligner  Bowtie2 $(pwd)/"$dir"_FU1_trim_up.fastq $(pwd)/"$dir"_FU2_trim_up.fastq; cd ..; done